In [2]:
import os
import pandas as pd

In [3]:
datasets_malware = [
    {"path": ["dataset/2017_dataset/Adware"], "class":"Adware"},
    {"path": ["dataset/2017_dataset/Benign"], "class":"Benign"},
    {"path": ["dataset/2017_dataset/Ransomware"], "class":"Ransomware"},
    {"path": ["dataset/2017_dataset/Scareware"], "class":"Scareware"},
    {"path": ["dataset/2017_dataset/SMSmalware"], "class":"SMSmalware"},
]

In [30]:
import dotenv
dotenv.load_dotenv()


True

In [2]:
abs_path = os.getenv('PATH_ROOT_DATASET')
# TODO: Columns that can be removed for column correlation analysis
# TODO: The initial version may change due to the relationship between the variables
columnTo_delete = ["Flow ID", "Source IP", "Destination IP", "Timestamp"]

In [9]:
dict_dataframes =  dict()
for metadata in datasets_malware:
    print(metadata['class'])
    dict_dataframes[metadata['class']] = pd.DataFrame()
    for item in metadata['path']:
        path_temp =  os.path.join(abs_path, item)
        abs =  os.path.abspath(path_temp)
        for root, dirs, files  in os.walk(abs): 
            for file in files:
                if file.endswith('.csv'):
                    path_data = os.path.join(root, file)
                    data =  pd.read_csv(path_data, index_col=False)
                    data.columns = data.columns.str.strip()
                    data.drop('Flow ID', axis=1, inplace=True)
                    dict_dataframes[metadata['class']] =  pd.concat([dict_dataframes[metadata['class']], data]) 

Adware
Benign
Ransomware
Scareware
SMSmalware


In [18]:
dict_dataframes['SMSmalware'].columns

Index(['Source IP', 'Source Port', 'Destination IP', 'Destination Port',
       'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packe

In [12]:
dict_dataframes_filter= {
    "RANSOMWARE": pd.DataFrame(),
    "ADWARE": pd.DataFrame(),
    "BENIGN": pd.DataFrame(),
    "SCAREWARE": pd.DataFrame(),
    "SMSMALWARE": pd.DataFrame()
}

dict_dataframes_invalid = dict()
labels = [
    "RANSOMWARE",
    "ADWARE",
    "BENIGN",
    "SCAREWARE",
    "SMSMALWARE",
]

In [13]:
for key,value in dict_dataframes.items():   
    incomplete_rows =  value[value.isnull().any(axis=1)]
    dict_dataframes_invalid[key] =  incomplete_rows
    complete_rows = value.dropna(subset=value.columns)
    for label in labels:
        filter_dataframe = complete_rows[complete_rows['Label'].str.contains(r""+label+r"", regex=True)]
        if len(filter_dataframe) > 0:
            dict_dataframes_filter[label] = pd.concat([dict_dataframes_filter[label], filter_dataframe])

In [14]:
dict_dataframes_filter['RANSOMWARE']

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,10.42.0.42,34528,172.217.12.174,443,6,29/08/2017 08:46:19,36753,1,1,23.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_SVPENG
1,10.42.0.42,58002,172.217.3.106,443,6,29/08/2017 08:46:20,37259,1,2,31.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_SVPENG
2,172.217.9.238,80,10.42.0.42,46352,6,29/08/2017 08:46:23,240329,2,0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_SVPENG
3,172.217.11.3,443,10.42.0.42,33677,6,29/08/2017 08:46:34,242945,2,0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_SVPENG
4,172.217.10.74,443,10.42.0.42,47775,6,29/08/2017 08:47:03,22,2,0,55.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_SVPENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179,10.42.0.151,60901,174.35.73.199,80,6,22/08/2017 09:06:57,23166,1,1,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
3180,10.42.0.151,8520,10.42.0.1,53,17,22/08/2017 09:06:53,637792,1,1,31.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
3181,10.42.0.151,34407,192.229.163.180,443,6,22/08/2017 09:56:40,8899330,2,0,0.0,...,32,0.0,0.0,0.0,0.0,8899330.0,0.0,8899330.0,8899330.0,RANSOMWARE_WANNALOCKER
3182,10.42.0.151,43477,111.221.77.172,40027,17,22/08/2017 09:53:51,228357,1,1,22.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER


In [35]:
def save_dic(dataframe, prefix=''):
    for key, value in dataframe.items():
        value.to_csv(f'{os.getenv("FOLDER_RESULT")}/{prefix}{key}.csv', index=False)

In [37]:
save_dic(dataframe=dict_dataframes_filter)
save_dic(dataframe=dict_dataframes_invalid,prefix='Invalid')